In [4]:
#!pip install neo4j

In [27]:
import os
import json
import pandas as pd
from py2neo import Graph,Node

class MedicalGraph:

    def read_nodes(self):

        departments = [] 
        diseases = [] 
        symptoms = []
        

        disease_infos = []

        rels_department = []

        rels_symptom = [] 
        rels_acompany = []
        rels_category = []


        fileName = "medical_knowledge_base.csv"
        df = pd.read_csv(fileName, encoding='gbk')
        count = 0
        for index, row in df.iterrows():
            disease_dict = {}
            count += 1
            print(count)
            disease = row['name']
            disease_dict['name'] = disease
            diseases.append(disease)
            disease_dict['desc'] = ''
            disease_dict['prevent'] = ''
            disease_dict['cause'] = ''
            disease_dict['easy_get'] = ''
            disease_dict['cure_department'] = ''
            disease_dict['cure_way'] = ''
            disease_dict['cure_lasttime'] = ''
            disease_dict['symptom'] = ''
            disease_dict['cured_prob'] = ''

            symtom_temp = row['symptom'].replace('[','').replace(']','').replace("'",'').split(",")
            symptoms += symtom_temp
            for symptom in symtom_temp:
                rels_symptom.append([disease, symptom])

            acompany_temp = row['acompany'].replace('[','').replace(']','').replace("'",'').split(",")
            for acompany in acompany_temp:
                rels_acompany.append([disease, acompany])

            disease_dict['desc'] = row['desc']

            disease_dict['prevent'] = row['prevent']

            disease_dict['cause'] = row['cause']


            cure_department = row['cure_department'].replace('[','').replace(']','').replace("'",'').split(",")
            if len(cure_department) == 1:
                 rels_category.append([disease, cure_department[0]])
            if len(cure_department) == 2:
                big = cure_department[0]
                small = cure_department[1]
                rels_department.append([small, big])
                rels_category.append([disease, small])

            disease_dict['cure_department'] = cure_department
            departments += cure_department

            disease_dict['cure_way'] = row['cure_way'].replace('[','').replace(']','').replace("'",'').split(",")

 
            disease_infos.append(disease_dict)
        return set(departments), set(symptoms), set(diseases), disease_infos,\
               rels_department, \
               rels_symptom, rels_acompany, rels_category

    def create_node(self, label, nodes):
        count = 0
        for node_name in nodes:
            node = Node(label, name=node_name)
            self.g.create(node)
            count += 1
            print(count, len(nodes))
        return

    def create_diseases_nodes(self, disease_infos):
        count = 0
        for disease_dict in disease_infos:
            node = Node("Disease", name=disease_dict['name'], desc=disease_dict['desc'],
                        prevent=disease_dict['prevent'] ,cause=disease_dict['cause'],
                        easy_get=disease_dict['easy_get'],cure_lasttime=disease_dict['cure_lasttime'],
                        cure_department=disease_dict['cure_department']
                        ,cure_way=disease_dict['cure_way'] , cured_prob=disease_dict['cured_prob'])
            self.g.create(node)
            count += 1
            print(count)
        return

    def create_graphnodes(self):
        Departments, Symptoms, Diseases, disease_infos, rels_department, rels_symptom, rels_acompany, rels_category = self.read_nodes()
        self.create_diseases_nodes(disease_infos)

        self.create_node('Department', Departments)
        print(len(Departments))

        self.create_node('Symptom', Symptoms)
        return


    def create_graphrels(self):
        Departments, Symptoms, Diseases, disease_infos, rels_department, rels_symptom, rels_acompany, rels_category = self.read_nodes()
        
        self.create_relationship('Department', 'Department', rels_department, 'belongs_to')
        self.create_relationship('Disease', 'Symptom', rels_symptom, 'has_symptom')
        self.create_relationship('Disease', 'Disease', rels_acompany, 'acompany_with')
        self.create_relationship('Disease', 'Department', rels_category, 'belongs_to')

    def create_relationship(self, start_node, end_node, edges, rel_type, rel_name):
        count = 0
        # 去重处理
        set_edges = []
        for edge in edges:
            set_edges.append('###'.join(edge))
        all = len(set(set_edges))
        for edge in set(set_edges):
            edge = edge.split('###')
            p = edge[0]
            q = edge[1]
            query = "match(p:%s),(q:%s) where p.name='%s'and q.name='%s' create (p)-[rel:%s{name:'%s'}]->(q)" % (
                start_node, end_node, p, q, rel_type, rel_name)
            try:
                self.g.run(query)
                count += 1
                print(rel_type, count, all)
            except Exception as e:
                print(e)
        return

    def export_data(self):
        Departments, Symptoms, Diseases, Cause, Acompany, Prevent, Lasttime, Cureway, Cureprob, Easyget, Desc, Getprob, Getway, disease_infos, rels_department, rels_symptom, rels_acompany, rels_category = self.read_nodes()
        f_department = open('dict/department.txt', 'w+')
        f_symptom = open('dict/symptoms.txt', 'w+')
        f_disease = open('dict/disease.txt', 'w+')


        f_department.write('\n'.join(list(Departments)))
        f_symptom.write('\n'.join(list(Symptoms)))
        f_disease.write('\n'.join(list(Diseases)))


        f_department.close()
        f_symptom.close()
        f_disease.close()


        return



if __name__ == '__main__':
    handler = MedicalGraph()
    handler.create_graphnodes()
    handler.create_graphrels()


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


AttributeError: 'MedicalGraph' object has no attribute 'g'